In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from scipy.spatial.distance import squareform
import scipy.cluster.hierarchy as sch #might be obsolete
from scipy.cluster.hierarchy import dendrogram, linkage
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import confusion_matrix
from function_pontius import *
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
%matplotlib inline

C:\Users\chimp\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
map_list = []
for i in range(2000):
    name = 'map' + str(i)
    map_list.append(name)

### Load Results ###
#Class 4 Industrial
dist_tca4 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA4_df.csv')
dist_tca4.index = map_list

dist_pland4 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND4_df.csv')
dist_pland4.index = map_list

#Class 14  Potatoes
dist_tca14 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/TCA14_df.csv')
dist_tca14.index = map_list

dist_pland14 = pd.read_csv('C:/LUMOS/MCK/Output_DFs/PLAND14_df.csv')
dist_pland14.index = map_list

#Class 22 Nature
dist_tca22 = pd.read_csv('E:/thesis_data/Output_DFs/Total Class Area[class 22]_df.csv')
dist_tca22.index = map_list
vals_tca22 = pd.read_csv('E:/thesis_data/Output_DFs/Total Class Area[class 22]_values.csv')

dist_pland22 = pd.read_csv('E:/thesis_data/Output_DFs/PLAND[class 22]_df.csv')
dist_pland22.index = map_list

# OQD_22
qd22_df = pd.read_csv('E:/thesis_data/Output_DFs/quantitydifferencecategorical_22_df.csv')
np.fill_diagonal(qd22_df.values, 0)

"""Non categorical metrics"""
# Shannon
dist_shan = pd.read_csv('E:/thesis_data/Output_DFs/shannon_df.csv')
dist_shan.index = map_list
# Simpsons
dist_simp = pd.read_csv('E:/thesis_data/Output_DFs/simp_df.csv')
dist_simp.index = map_list
# Kappa
kappa_df = pd.read_csv('E:/thesis_data/Output_DFs/kappa.csv', index_col= 'Unnamed: 0')
dist_kappa = 1 - kappa_df
# OA
oa_df = pd.read_csv('E:/thesis_data/Output_DFs/overallaccuracy_df.csv')
dist_oa = 1 - oa_df
dist_oa.index = map_list
# OAD
oad_df = pd.read_csv('E:/thesis_data/Output_DFs/overallallocationdifference_df.csv')
oad_df.index = map_list
np.fill_diagonal(oad_df.values, 0)
# OD
od_df = pd.read_csv('E:/thesis_data/Output_DFs/overalldifference_df.csv')
od_df.index = map_list
np.fill_diagonal(od_df.values, 0)
# OQD
oqd_df = pd.read_csv('E:/thesis_data/Output_DFs/overallquantitydifference_df.csv')
oqd_df.index = map_list
np.fill_diagonal(oqd_df.values, 0)

In [3]:
def find_medoids(df, metric, nr_clusters):
    med_list = []
    for i in range(nr_clusters):
        med_df = find_cluster(df, metric, i) 
        med_map = med_df['Sum'].idxmin(axis=0, skipna=True)
        med_list.append(med_map)
    return med_list
    
def find_cluster(df=dist_kappa, metric='kappa', cluster=0):
    global df_cluster
    metric_list= ['kappa', 'oa', 'shan', 'simp', 'od', 'oad', 'oqd', 'tca4', 'tca14', 'tca22', 'pland4', 'pland14', 'pland22', 'qd22']
    metric_list.remove(metric)
    df_clean = df_cluster[df_cluster[metric] == cluster].drop(axis='columns', labels=metric_list)
    reduc_list = df_clean.index
    df_reduc = df
    df_reduc = df_reduc[df_reduc.index.isin(reduc_list)]
    df_reduc = df_reduc[reduc_list]
    df_reduc['Sum'] = df_reduc.sum(axis='index')
    return df_reduc

In [4]:
"""Cluster with the found number of clusters"""
kappaclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(dist_kappa)
shannonclusters = AgglomerativeClustering(n_clusters=4, affinity='precomputed', linkage='complete').fit(dist_shan)
simpsonclusters = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_simp)
oaclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(dist_oa)
odclusters = AgglomerativeClustering(n_clusters=7, affinity='precomputed', linkage='complete').fit(od_df)
oadclusters = AgglomerativeClustering(n_clusters=6, affinity='precomputed', linkage='complete').fit(oad_df)
oqdclusters = AgglomerativeClustering(n_clusters=4, affinity='precomputed', linkage='complete').fit(oqd_df)
qd22clusters = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(qd22_df)
tcacluster4 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_tca4)
tcacluster14 = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_tca14)
tcacluster22 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_tca22)
plandcluster4 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_pland4)
plandcluster14 = AgglomerativeClustering(n_clusters=5, affinity='precomputed', linkage='complete').fit(dist_pland14)
plandcluster22 = AgglomerativeClustering(n_clusters=3, affinity='precomputed', linkage='complete').fit(dist_pland22)

labels_kappa = kappaclusters.labels_
labels_shan = shannonclusters.labels_
labels_simp = simpsonclusters.labels_
labels_oa = oaclusters.labels_
labels_od = odclusters.labels_
labels_oad = oadclusters.labels_
labels_oqd = oqdclusters.labels_
labels_qd22 = qd22clusters.labels_
labels_tca4 = tcacluster4.labels_
labels_tca14 = tcacluster14.labels_
labels_tca22 = tcacluster22.labels_
labels_pland4 = plandcluster4.labels_
labels_pland14 = plandcluster14.labels_
labels_pland22 = plandcluster22.labels_

In [5]:
df_cluster = pd.DataFrame(index = map_list)
df_cluster['kappa'] = labels_kappa
df_cluster['oa'] = labels_oa
df_cluster['shan'] = labels_shan
df_cluster['simp'] = labels_simp
df_cluster['od'] = labels_od
df_cluster['oad'] = labels_oad
df_cluster['oqd'] = labels_oqd
df_cluster['tca4'] = labels_tca4
df_cluster['tca14'] = labels_tca14
df_cluster['tca22'] = labels_tca22
df_cluster['pland4'] = labels_pland4
df_cluster['pland14'] = labels_pland14
df_cluster['pland22'] = labels_pland22
df_cluster['qd22'] = labels_qd22

df_cluster.head(5)

,kappa,oa,shan,simp,od,oad,oqd,tca4,tca14,tca22,pland4,pland14,pland22,qd22
map0,2,2,0,1,1,1,0,2,2,2,2,3,2,2
map1,6,6,3,4,2,5,2,1,1,0,1,4,0,0
map2,1,1,0,0,4,3,1,0,4,0,0,1,0,0
map3,3,6,1,2,3,0,2,1,1,0,1,2,0,0
map4,6,6,2,4,2,1,2,1,1,2,1,4,2,2


In [6]:
meds_kappa = sorted(find_medoids(dist_kappa, 'kappa', 7), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_oa = sorted(find_medoids(dist_oa, 'oa', 7), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_shan = sorted(find_medoids(dist_shan, 'shan', 4), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_simp = sorted(find_medoids(dist_simp, 'simp', 5), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_od = sorted(find_medoids(od_df, 'od', 7), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_oad = sorted(find_medoids(oad_df, 'oad', 6), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_oqd = sorted(find_medoids(oqd_df, 'oqd', 4), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_qd22 =  sorted(find_medoids(qd22_df, 'qd22', 3), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_tca4 = sorted(find_medoids(dist_tca4, 'tca4', 3), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_tca14 = sorted(find_medoids(dist_tca14, 'tca14', 5), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_tca22 = sorted(find_medoids(dist_tca22, 'tca22', 3), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_pland4 = sorted(find_medoids(dist_pland4, 'pland4', 3), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_pland14 = sorted(find_medoids(dist_pland14, 'pland14', 5), key=lambda x: int("".join([i for i in x if i.isdigit()])))
meds_pland22 = sorted(find_medoids(dist_pland22, 'pland22', 3), key=lambda x: int("".join([i for i in x if i.isdigit()])))

In [7]:
"""Make all medoid map lists length 7 so they can be used to make a df """
list_of_meds = [meds_kappa, meds_oa, meds_shan, meds_simp, meds_od, meds_oad, meds_oqd, meds_qd22, meds_tca4, meds_tca14, meds_tca22, meds_pland4, meds_pland14, meds_pland22]
def list_filler(entry):
    if len(entry) < 7:
        fillers = 7 - len(entry)
        for i in range(fillers):
            entry.append('NA')
            
for entry in list_of_meds:
    list_filler(entry)

In [8]:
df_med = pd.DataFrame({'kappa': meds_kappa,
     'oa': meds_oa,
     'shan': meds_shan,
     'simp': meds_simp,
     'od': meds_od,
     'oad': meds_oad,
     'oqd': meds_oqd,    
     'tca4': meds_tca4,
     'pland4': meds_pland4,                       
     'tca14': meds_tca14,
     'pland14': meds_pland14,                       
     'tca22': meds_tca22,  
     'pland22': meds_pland22,
     'qd22' : meds_qd22
    })
df_med

,kappa,oa,shan,simp,od,oad,oqd,tca4,pland4,tca14,pland14,tca22,pland22,qd22
0,map359,map10,map61,map65,map359,map263,map1057,map217,map264,map246,map246,map0,map0,map0
1,map456,map359,map1248,map857,map456,map453,map1472,map332,map1422,map296,map325,map8,map8,map1
2,map541,map456,map1454,map1448,map541,map678,map1833,map604,map1877,map325,map564,map35,map35,map8
3,map695,map541,map1457,map1784,map695,map1760,map1978,NA,NA,map825,map704,NA,NA,NA
4,map1148,map695,NA,map1830,map1148,map1852,NA,NA,NA,map1612,map843,NA,NA,NA
5,map1760,map696,NA,NA,map1760,map1918,NA,NA,NA,NA,NA,NA,NA,NA
6,map1911,map1911,NA,NA,map1911,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [ ]:
0 8 35